In [3]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
# from ggplot import *


from ruffus import *
import sys, os, h5py, random, tempfile, scipy, time,copy
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.parse_gct as parse_gct
import pipeline_support as PS
from sklearn.decomposition import PCA
from scipy import stats
from matplotlib.pyplot import imshow
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
#from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow_gan.python.losses import losses_impl


from numpy.random import seed


import scipy.stats as ss
import warnings
import numpy as np
from maayanlab_bioinformatics.normalization import quantile_normalize
import numpy_indexed as npi

randomState = 123
seed(randomState)

input filename

In [4]:
l1000_filename = "../data/L1000/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx"
l1000_sampleinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_inst_info.txt"
l1000_geneinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_gene_info.txt"

geo_id_dict_filename = "../data/Evaluation/MetaSRA/geo_id_dict.pkl"

ARCHS4_filename = "../data/ARCHS4/human_matrix_v9.h5"
ARCHS4_filtered_sample_output_filename = "../data/processed/ARCHS4/filtered_sample_list.txt"

l1000_all_gene_list = "../data/L1000/all_gene_list.txt"
l1000_landmark_gene_list = "../data/L1000/landmark_gene_list.txt"

archs4_all_gene_list = "../data/ARCHS4/all_gene_list.txt"
archs4_high_count_gene_list = "../data/ARCHS4/high_count_gene_list.txt"

gtex_l1000_all_gene_list = "../data/GTEx/l1000_all_gene_list.txt"
gtex_l1000_landmark_gene_list = "../data/GTEx/l1000_landmark_gene_list.txt"

gtex_rnaseq_all_gene_list = "../data/GTEx/rnaseq_all_gene_list.txt"

output filename

In [5]:
l1000_output_filename = "../data/Evaluation/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n{}x{}_cellline{}.f"
archs4_output_filename = "../data/Evaluation/ARCHS4_human_matrix_v9_n{}x{}_cellline{}.f"

## Get overlap_landmark_file

In [6]:
with open(l1000_landmark_gene_list, "r") as f:
    l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(archs4_all_gene_list, "r") as f:
    archs4_all_gene = [x.strip() for x in f.readlines()]
with open(gtex_l1000_landmark_gene_list, "r") as f:
    gtex_l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(gtex_rnaseq_all_gene_list, "r") as f:
    gtex_rnaseq_all_gene = [x.strip() for x in f.readlines()]
    



In [7]:
overlap_landmark_genes = list(set(l1000_landmark_gene).intersection(archs4_all_gene).intersection(gtex_l1000_landmark_gene).intersection(gtex_rnaseq_all_gene))

# Load Genes with high counts

In [9]:
with open(archs4_high_count_gene_list, "r") as f:
    archs4_high_count_genes = [x.strip() for x in f.readlines()]

# Load GEO ID list

In [10]:
with open(geo_id_dict_filename, "rb") as f:
    geo_id_dict = pickle.load(f)

# Load L1000 data

In [17]:
print('Loading L1000 data.....')
l1000_data = parse_gctx.parse(l1000_filename,convert_neg_666=True).data_df

Loading L1000 data.....


In [18]:
sample_info = pd.read_csv(l1000_sampleinfo_filename, sep="\t")

/home/maayanlab/anaconda3/envs/python37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
gene_info = pd.read_csv(l1000_geneinfo_filename,header = 0, sep = '\t')

In [20]:
# create a probe_id to gene name dictionary 
gene_dict = dict(zip([str(x) for x in gene_info['pr_gene_id']], gene_info['pr_gene_symbol']))

# label rows with gene names 
l1000_data.index = [gene_dict[x] for x in l1000_data.index.values]


In [21]:
# filter landmark genes
filtered_l1000_data = l1000_data[l1000_data.index.isin(overlap_landmark_genes)]

In [22]:
del l1000_data

# Load ARCHS4

In [11]:
# Import ARCHS4 RNA-seq samples 
print('Processing RNA-seq data.....')
h5 = h5py.File(ARCHS4_filename, 'r')
data_file = h5['data'] 
expression = data_file['expression']
genes = [x for x in h5['meta']['genes']['genes']]
sample_geo_list = list(h5['meta']['samples']['geo_accession'])

Processing RNA-seq data.....


In [12]:
with open(ARCHS4_filtered_sample_output_filename, "r") as f:
    filtered_sample_ids = [x.strip() for x in f.readlines()] #~150K

In [13]:
def get_archs4_samples(sampled_ids):
    # index of selected samples
    sampled_index = npi.indices(sample_geo_list, sampled_ids)
    sample_index_to_id_dict = dict(zip(sampled_index, sampled_ids))
    
    expression_i = expression[:, sorted(sampled_index)]
    expression_i_df = pd.DataFrame(expression_i)
    expression_i_df.columns = [sample_index_to_id_dict[k] for k in sorted(sampled_index)]
    
    # 35238 -> 25312
    expression_i_df.loc[archs4_high_count_genes, :]
    return expression_i_df

# Normalization

In [14]:
def CPM(data):

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = (data/data.sum())*10**6
        data = data.fillna(0)
        
    return data
def logCPM(data):

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = (data/data.sum())*10**6
        data = data.fillna(0)
        data = np.log10(data+1)

    # Return
    return data
def log(data):

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data = data.fillna(0)
        data = np.log10(data+1)

    return data

def rpkm(counts, lengths):
    """Calculate reads per kilobase transcript per million reads.

    RPKM = (10^9 * C) / (N * L)

    Where:
    C = Number of reads mapped to a gene
    N = Total mapped reads in the experiment
    L = Exon length in base pairs for a gene

    Parameters
    ----------
    counts: array, shape (N_genes, N_samples)
        RNAseq (or similar) count data where columns are individual samples
        and rows are genes.
    lengths: array, shape (N_genes,)
        Gene lengths in base pairs in the same order
        as the rows in counts.

    Returns
    -------
    normed : array, shape (N_genes, N_samples)
        The RPKM normalized counts matrix.
    """
    N = np.sum(counts, axis=0)  # sum each column to get total reads per sample
    L = lengths
    C = counts

    normed = 1e9 * C / (N[np.newaxis, :] * L[:, np.newaxis])

    return(normed)
    
def qnormalization(data):

    X_quantile_norm = quantile_normalize(data)
    return X_quantile_norm  

def normalization(data, logCPM_normalization=False, CPM_normalization=False, log_normalization=False, z_normalization=False, q_normalization=False):
    if logCPM_normalization == True:  
        data = logCPM(data)
    if CPM_normalization == True:
        data = CPM(data)
    if log_normalization == True:   
        data = log(data)
        
    if q_normalization == True:
        data = qnormalization(data)
        
    
    if z_normalization == True: 
        data = data.T.apply(ss.zscore, axis=0).T.dropna()

    return data

In [26]:
def get_archs4_samples(sampled_ids):
    sampled_ids = sorted(sampled_ids)
    # index of selected samples
    common_sample_ids = sorted(list(set(sampled_ids).intersection(set(sample_geo_list))))
    
    sampled_index = npi.indices(sample_geo_list, common_sample_ids)
    
    sample_index_to_id_dict = dict(zip(sampled_index, common_sample_ids))
    
    expression_i = expression[:, sorted(sampled_index)]
    expression_i_df = pd.DataFrame(expression_i)
    expression_i_df.columns = [sample_index_to_id_dict[k] for k in sorted(sampled_index)]
    expression_i_df.index = genes
    
    expression_i_df = expression_i_df.loc[archs4_high_count_genes, :]
    
    return expression_i_df

In [39]:
for cell_line in geo_id_dict.keys():
    print(cell_line)
    cell_line_sample_info = sample_info[(sample_info["pert_id"] == "DMSO") & (sample_info["cell_id"]==cell_line)]
    
    geo_id_dict[cell_line] = [x for x in geo_id_dict[cell_line] if x != ""]
    
    # get L1000 for the cell line
    filtered_l1000_data_cellline = filtered_l1000_data.loc[:, cell_line_sample_info["inst_id"].tolist()].T

    
    # get RNA-seq for the cell line
    filtered_archs4_data_cellline = get_archs4_samples(geo_id_dict[cell_line]).T
    print(filtered_archs4_data_cellline.shape)
    
    # sampling L1000 the same number as ARCHS4
    filtered_l1000_data_cellline_sampling = filtered_l1000_data_cellline.sample(filtered_archs4_data_cellline.shape[0])
    
    # normalize
    filtered_l1000_data_cellline_normalized = normalization(filtered_l1000_data_cellline_sampling.T, q_normalization=True).T
    filtered_archs4_data_cellline_normalized = normalization(filtered_archs4_data_cellline.T, logCPM_normalization=True, q_normalization=True).T
    
    # landmark genes
    filtered_archs4_data_cellline_normalized_landmark = filtered_archs4_data_cellline_normalized.loc[overlap_landmark_genes, :]
    # save
    filtered_l1000_data_cellline_normalized.reset_index().to_feather(l1000_output_filename.format(filtered_l1000_data_cellline_normalized.shape[0], filtered_l1000_data_cellline_normalized.shape[1], cell_line))
    filtered_archs4_data_cellline_normalized.reset_index().to_feather(archs4_output_filename.format(filtered_archs4_data_cellline_normalized.shape[0], filtered_archs4_data_cellline_normalized.shape[1], cell_line))
    filtered_archs4_data_cellline_normalized_landmark.reset_index().to_feather(archs4_output_filename.format(filtered_archs4_data_cellline_normalized_landmark.shape[0], filtered_archs4_data_cellline_normalized_landmark.shape[1]))
#     print(filtered_l1000_data_cellline_normalized)
#     print(filtered_archs4_data_cellline_normalized)
    
#     break

MCF7
(203, 25312)
PC3
(31, 25312)
A375
(30, 25312)
HEPG2
(7, 25312)
VCAP
(4, 25312)


In [32]:
filtered_archs4_data_cellline

,A1BG,A1CF,A2M,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,...,ZYX,ZZEF1,ZZZ3,BP-21201H5.1,BP-21264C1.1,BP-2171C21.2,BP-2171C21.4,BP-2171C21.5,BP-2171C21.6,BP-2189O9.2
GSM1244820,65,5,10,6,13,0,37,0,1466,2147,...,3364,2716,2631,34,51,0,5,0,5,18
GSM1069746,46,7,2,39,17,0,93,0,514,1174,...,1307,1323,705,7,25,0,0,2,2,7
GSM942209,160,3,18,50,25,1,415,0,1403,2958,...,1583,5024,2236,22,77,0,1,0,9,22
GSM1244818,63,0,3,9,16,0,42,0,1620,2278,...,3275,3072,2149,38,52,1,0,0,13,13
GSM1244822,105,9,18,19,21,0,57,0,2970,4256,...,6205,5423,4729,58,108,1,6,0,25,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3538846,84,3,1,69,1,0,950,0,1334,1340,...,605,1837,2668,0,8,0,0,1,2,9
GSM4081348,101,0,10,1,3,0,457,0,528,501,...,755,564,177,6,0,0,0,0,1,2
GSM4081349,54,0,1,4,2,0,284,0,320,308,...,441,482,390,6,4,0,0,0,0,3
GSM4081352,165,5,16,7,3,0,820,0,1148,1218,...,1162,1597,627,15,1,0,1,4,11,6


In [37]:
(filtered_archs4_data_cellline == 0).astype(int).sum(axis=1)

GSM1244820    5163
GSM1069746    5878
GSM942209     4883
GSM1244818    5159
GSM1244822    4066
              ... 
GSM3538846    6330
GSM4081348    8235
GSM4081349    8525
GSM4081352    6722
GSM4081353    8141
Length: 203, dtype: int64

In [30]:
filtered_archs4_data_cellline_normalized

,A1BG,A1CF,A2M,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,...,ZYX,ZZEF1,ZZZ3,BP-21201H5.1,BP-21264C1.1,BP-2171C21.2,BP-2171C21.4,BP-2171C21.5,BP-2171C21.6,BP-2189O9.2
GSM1244820,0.499148,0.077632,0.146216,0.091057,0.181177,0.000000,0.359592,0.0,1.666141,1.830923,...,2.026383,1.929692,1.914497,0.342176,0.432671,0.000000,0.077632,0.000000,0.077632,0.232676
GSM1069746,0.624828,0.158088,0.042253,0.568131,0.324081,0.000000,0.904839,0.0,1.612272,1.958203,...,2.001709,2.007102,1.742260,0.158088,0.427495,0.000000,0.000000,0.042253,0.042253,0.158088
GSM942209,0.740817,0.033552,0.184026,0.393107,0.240254,0.005784,1.094547,0.0,1.618791,1.941365,...,1.668504,2.165832,1.819612,0.217411,0.506358,0.000000,0.005784,0.000000,0.102093,0.217411
GSM1244818,0.484494,0.000000,0.043525,0.127860,0.209343,0.000000,0.385393,0.0,1.724014,1.865199,...,2.023462,1.996412,1.838877,0.366721,0.437361,0.008121,0.000000,0.000000,0.174027,0.174027
GSM1244822,0.450148,0.066832,0.130643,0.138360,0.150356,0.000000,0.309904,0.0,1.684764,1.836747,...,2.004296,1.941199,1.883010,0.313327,0.456325,0.002042,0.044851,0.000000,0.173280,0.219988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3538846,0.847936,0.088696,0.020267,0.789552,0.020267,0.000000,1.800242,0.0,1.941654,1.944572,...,1.614222,2.083172,2.243796,0.000000,0.221667,0.000000,0.000000,0.020267,0.054719,0.244752
GSM4081348,1.266858,0.000000,0.494607,0.062505,0.203678,0.000000,1.810808,0.0,1.864541,1.843592,...,1.999887,1.888118,1.464791,0.351655,0.000000,0.000000,0.000000,0.000000,0.062505,0.138323
GSM4081349,1.009861,0.000000,0.072061,0.270031,0.149714,0.000000,1.619484,0.0,1.667667,1.650962,...,1.798051,1.837976,1.745063,0.367714,0.270031,0.000000,0.000000,0.000000,0.000000,0.217294
GSM4081352,1.119146,0.160137,0.398369,0.216250,0.101222,0.000000,1.702184,0.0,1.825903,1.848465,...,1.831288,1.956366,1.598800,0.382460,0.027996,0.000000,0.027996,0.131158,0.309075,0.189340


In [38]:
geo_id_dict.keys()

dict_keys(['MCF7', 'PC3', 'A375', 'HEPG2', 'VCAP'])